In [1]:
import time 
import json # manipulación de archivos json
import os # funciones del sistema operativo
import logging # escribir logs
import requests # requests de páginas
from threading import Thread
from http.client import IncompleteRead
from twython import TwythonStreamer # Obtener los tweets en tiempo real
from twython import Twython # Generar conexión con Twitter mediante Twithon

## Se accederán a los tweets en tiempo real mediante la API Twython
## Se contará con un archivo config en formato JSON que contendrá los parámetros relevantes para las conexiones y búsquedas

In [2]:
"""" 
Declaro una clase Authentication cuyas instancias permitirán leer el archivo config para obtener los parámetros
de conexión y de búsqueda de contenidos en tweets, como así también de almacenamiento de archivos json.
"""
class Authentication:
    
    """
    Función que me permite recorrer un archivo JSON, en donde iremos recorriendo línea por línea para obtener
    el contenido de cada atributo y devolverlos como resultado
    """
    def read_config_file(self, filename):
        with open(filename, "r") as f:
            s = f.read()
        d = json.loads(s)
        APP_KEY = d["APP_KEY"]
        APP_SECRET = d["APP_SECRET"]
        TERMS_FILE = d["TERMS_FILE"]
        STORAGE_PATH = d["STORAGE_PATH"]
        return APP_KEY, APP_SECRET, TERMS_FILE, STORAGE_PATH
    
    """
    Obtengo los accesos mediante los keys de Twitter
    """
    def get_oauth_link(self, APP_KEY, APP_SECRET):
        twitter = Twython(APP_KEY, APP_SECRET) # genero un objeto de la API
        auth = twitter.get_authentication_tokens() # obtengo los tokens de autenticación
        # guardo los valores obtenidos en variables
        OAUTH_TOKEN = auth['oauth_token']
        OAUTH_TOKEN_SECRET = auth['oauth_token_secret']
        url = (auth['auth_url'])
        # genero un requests a la url obtenida
        r = requests.get(url)
        # muestro el link de twitter para que genere el pin de autorización a la app
        logging.info("Dirigirse al siguiente link, generar y copiar el pin en el archivo 'code.txt':")
        logging.info(r.url)
        return url, OAUTH_TOKEN, OAUTH_TOKEN_SECRET

    """
    Esta función genera una espera hasta encontrar el código correcto en el archivo 'code.txt'
    Una vez cuen encuetra el código, lo retorna como resultado de la función.
    """
    def wait_for_pin_code(self):
        while True:
            if not os.path.exists("code.txt"):
                time.sleep(5)
                logging.debug("'code.txt' no existe, esperando el código en el archivo para empezar a escuchar Tweets"
                )
            else:
                pincode = 0
                with open("code.txt") as f:
                    pincode = int(f.read().strip())
                    logging.info("Código leído con éxito:" + str(pincode))
                return str(pincode)
            
    """
    Función que elimina el archivo 'code.txt'
    """
    def remove_old_code_file(self):
        if os.path.exists("code.txt"):
            os.remove("code.txt")
    
    """
    Función que genera la conexión a la API con autorización del código o pin
    """
    def auth_with_pin(self, APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET, pincode):
        # genero la instancia de twitter usando Twithon
        twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
        final_step = twitter.get_authorized_tokens(pincode)
        logging.debug("Anterior OATH_TOKEN: " + str(OAUTH_TOKEN))
        logging.debug("Anterior OAUTH_TOKEN_SECRET: " + str(OAUTH_TOKEN_SECRET))
        OAUTH_TOKEN = final_step['oauth_token']
        OAUTH_TOKEN_SECRET = final_step['oauth_token_secret']
        logging.debug("Nuevo OATH_TOKEN: " + str(OAUTH_TOKEN))
        logging.debug("Nuevo OAUTH_TOKEN_SECRET: " + str(OAUTH_TOKEN_SECRET))
        return OAUTH_TOKEN, OAUTH_TOKEN_SECRET

In [3]:
# Clase que define una excepción 
class TooLongTermException(Exception):
    
    def __init__(self, index):
        self.index = index

    def get_too_long_index(self):
        return self.index

In [4]:
"""
Clase que permite buscar los tweets en tiempo real, para lo cual se herda de la clase TwithonStreamer
"""
class StreamListener(TwythonStreamer):
    
    # Genera una instancia de la clase
    def __init__(self, APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET, comm_list):
        #inicializa la instancia invocando al constructor de la clase padre
        super().__init__(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
        #define un atributo propio llamado tweet_list
        self.tweet_list = comm_list

    # Redefino la función on_success en caso de que obtenga información y la agrega a la lista  
    def on_success(self, data):
        self.tweet_list.append(data)
        logging.info("tweet capturado")
    
    # Redefino la función on_error encaso
    # todo status_code distinto de 200 es un error
    def on_error(self, status_code, data):
        logging.error(status_code)
        logging.error(data)
        if int(status_code) == 406:
            data = str(data)
            try:
                index = int(data.strip().split()[4])
                logging.error("to remove index:" + str(index))
                raise TooLongTermException(index)
            except ValueError:
                logging.debug("ValueError while trying to extract number")

In [5]:
def get_authentication():
    
    # Defino una instancia de Authentication
    auth = Authentication()
    
    # Defino del formato del log para que muestre la fecha y hora del evento
    logging.basicConfig(
        format='%(levelname)s: %(asctime)s - %(message)s',
        datefmt='%m/%d/%Y %I:%M:%S %p',
        level=logging.INFO
    )

    logging.info("Borrando el archivo code.txt")
    auth.remove_old_code_file()

    logging.info("Obteniendo información del archivo config.json")
    APP_KEY, APP_SECRET, TERMS_FILE, STORAGE_PATH = auth.read_config_file("config.json")

    logging.info("Obteniendo las claves de autorización")
    url, OAUTH_TOKEN, OAUTH_TOKEN_SECRET = auth.get_oauth_link(APP_KEY, APP_SECRET)

    logging.info("Esperando el código en code.txt")
    pincode = auth.wait_for_pin_code()

    logging.info("Obteniendo autorización con el código")
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET = auth.auth_with_pin(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET, pincode)

    logging.info("Comienzo de lectura de tweets....")

    filter_terms = []
    with open(TERMS_FILE) as f:
        for term in f:
            filter_terms.append(term.strip())
    logging.info("Los términos a buscar en los tweets son " + str(filter_terms))

    return APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET, filter_terms, STORAGE_PATH

In [6]:
# Defino una función para obtener los tweets
def twitter_listener(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET, comm_list):
    
    # Instancio la clase StreamListener
    streamer = StreamListener(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET, comm_list)
    
    while True:
        try:
            # Con la instancia generada filtro los tweets de acuerdo a los términos seteados
            streamer.statuses.filter(track=[', '.join(filter_terms)], language='es')
        except requests.exceptions.ChunkedEncodingError:
            print('Error. Continúa...\n')
            pass
        except IncompleteRead:
            print('Lectura incompleta. Continúa...')
            pass
        except TooLongTermException as e:
            index_to_remove = e.get_too_long_index()
            filter_terms.pop(index_to_remove)

# Defino una función para escribir los tweets encontrados
def twitter_writer(comm_list):
    internal_list = []
    time_start = time.time()
    while True:
        if len(internal_list) > 100:
            file_name = STORAGE_PATH + str(round(time.time())) + ".json"
            with open(file_name, 'w+', encoding='utf-8') as output_file:
                json.dump(internal_list, output_file, indent=4)
                internal_list = []
                logging.info('------- Data dumped -------')
                time_stop = time.time()
                logging.info('Tiempo transcurrido para obtener 100 tweets: {0:.2f}s'.format(time_stop - time_start))
                time_start = time.time()
        else:
            for i in range(len(comm_list)):
                internal_list.append(comm_list.pop())
            time.sleep(1)

In [ ]:
if __name__ == '__main__':
    
    # Obtengo las autenticaciones y parámetros de la captura de tweets
    APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET, filter_terms, STORAGE_PATH = get_authentication()
    comm_list =[]
    
    # Se genera un hilo pasando como target la función que leerá los tweets
    listener = Thread(target = twitter_listener, args = (APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET, comm_list ))
    listener.start()
    writer = Thread(target = twitter_writer, args = (comm_list,))
    writer.start()
    writer.join()
    listener.join()

INFO: 04/04/2019 08:02:16 PM - Borrando el archivo code.txt
INFO: 04/04/2019 08:02:16 PM - Obteniendo información del archivo config.json
INFO: 04/04/2019 08:02:16 PM - Obteniendo las claves de autorización
INFO: 04/04/2019 08:02:17 PM - Dirigirse al siguiente link, generar y copiar el pin en el archivo 'code.txt':
INFO: 04/04/2019 08:02:17 PM - https://api.twitter.com/oauth/authorize?oauth_token=zlPBfAAAAAAA9rRDAAABaeqWpcc
INFO: 04/04/2019 08:02:17 PM - Esperando el código en code.txt
INFO: 04/04/2019 08:02:47 PM - Código leído con éxito:4389286
INFO: 04/04/2019 08:02:47 PM - Obteniendo autorización con el código
INFO: 04/04/2019 08:02:47 PM - Comienzo de lectura de tweets....
INFO: 04/04/2019 08:02:47 PM - Los términos a buscar en los tweets son ['Kirchner', 'Macri', 'Lavagna']
INFO: 04/04/2019 08:02:49 PM - tweet capturado
INFO: 04/04/2019 08:02:49 PM - tweet capturado
INFO: 04/04/2019 08:02:50 PM - tweet capturado
INFO: 04/04/2019 08:02:50 PM - tweet capturado
INFO: 04/04/2019 08:0

INFO: 04/04/2019 08:04:24 PM - tweet capturado
INFO: 04/04/2019 08:04:25 PM - tweet capturado
INFO: 04/04/2019 08:04:26 PM - tweet capturado
INFO: 04/04/2019 08:04:27 PM - tweet capturado
INFO: 04/04/2019 08:04:27 PM - tweet capturado
INFO: 04/04/2019 08:04:29 PM - tweet capturado
INFO: 04/04/2019 08:04:29 PM - tweet capturado
INFO: 04/04/2019 08:04:30 PM - tweet capturado
INFO: 04/04/2019 08:04:30 PM - tweet capturado
INFO: 04/04/2019 08:04:31 PM - tweet capturado
INFO: 04/04/2019 08:04:31 PM - tweet capturado
INFO: 04/04/2019 08:04:31 PM - tweet capturado
INFO: 04/04/2019 08:04:34 PM - tweet capturado
INFO: 04/04/2019 08:04:34 PM - tweet capturado
INFO: 04/04/2019 08:04:34 PM - tweet capturado
INFO: 04/04/2019 08:04:35 PM - tweet capturado
INFO: 04/04/2019 08:04:37 PM - tweet capturado
INFO: 04/04/2019 08:04:38 PM - tweet capturado
INFO: 04/04/2019 08:04:38 PM - tweet capturado
INFO: 04/04/2019 08:04:38 PM - tweet capturado
INFO: 04/04/2019 08:04:39 PM - tweet capturado
INFO: 04/04/2